In [1]:
import pygame
import math
from queue import PriorityQueue # 根據前面 index 優先順序出 value

# Setting the size of display we will monitoring.
WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("The A* Path Finding Algorithm")

pygame 2.0.1 (SDL 2.0.14, Python 3.6.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Setting the RGB code we will use
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE =(0, 0, 255)
YELLOW =(255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
ORANGE = (255, 165, 0)
GRAY = (128, 128, 128)
TURQUOISE = (64, 224, 208) 

In [3]:
#Define the spot
class Spot:
    #Constructor
    def __init__(self, row, col, width, total_rows):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.color = WHITE
        self.neighbors = [] # The concept of 'neighbors' is as similiar with Dijkstra's algorithm
        self.width = width  # The size of spot
        self.total_rows = total_rows 
        
    def get_pos(self):
        return self.col, self.col
    
    #Set the displayed color to represent the current state of this point
    def is_start(self):
        return self.color == ORANGE
    def is_end(self):
        return self.color == PURPLE    
    def is_barrier(self):
        return self.color == BLACK
    
    #Set the displayed color to represent whether the current point has been judged 
    def is_closed(self):
        return self.color == RED #Python裡面 "== 為比較是否相等
    def is_open(self):
        return self.color == GREEN
            
    
    def reset(self):
        self.color = WHITE #Python裡面 "=" 為賦予值, updating目前的value
    def make_start(self):
        self.color = ORANGE
    def make_closed(self):
        self.color = RED
    def make_open(self):
        self.color = GREEN
    def make_barrier(self):
        self.color = BLACK
    def make_end(self):
        self.color = TURQUOISE
    def make_path(self):
        self.color = PURPLE

    def draw(self, win):                  #Prepare the position to be drawn
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))
    def update_neighbors(self, grid):
        self.neighbors = []
        #if the row is within total rows, and the grid is not barrier
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier(): # DOWN (由上往下走)
            #specify this current grid as one of neighbors of spot
            self.neighbors.append(grid[self.row + 1][self.col])

        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier(): # UP (由下往上走)
            self.neighbors.append(grid[self.row - 1][self.col])
        #if the col is within total cols, and the grid is not barrier
        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier(): # RIGHT (由左往右走)
            self.neighbors.append(grid[self.row][self.col + 1])

        if self.col > 0 and not grid[self.row][self.col - 1].is_barrier(): # LEFT (由右往左走)
            self.neighbors.append(grid[self.row][self.col - 1])

In [4]:
#### heuristic function ####
def h(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2) # 曼哈顿距离(Manhattan Distance)

# The main algorithm
def algorithm(draw, grid, start, end):
    count = 0
    #Setting a set called "open_set" for collecting the grid prepared to process
    open_set = PriorityQueue()
    open_set.put((0, count, start)) #add the "start" node with its f score into the "open_set"
    came_from = {}
    
    #Initialization the f_score in whole table, specified all the grid as inf
    f_score = {spot: float("inf") for row in grid for spot in row}        
    #Specified the f_score of "start" and "end" by Manhattan Distance 指定start點的 f_score 為 start和 end 兩點的曼哈頓距離
    f_score[start] = h(start.get_pos(), end.get_pos()) #<=== 初步為點start和end點的曼哈頓距離
    #Initialization the g_score in whole table, specified all the grid as inf
    g_score = {spot: float("inf") for row in grid for spot in row}
    #Specified the g_score of "start" as 0 指定 start點的 g_score 為 0
    g_score[start] = 0
   
    """
    Putting "start" node into open_set_hash set, the reason is the PriorityQueue() doesn't actually have anything to 
    tell us if a node is in the queue or not, so we need to set hash that is going to keep track of all the items whether 
    are in the PriorityQueue() 
    """
    open_set_hash = {start}

    #When the "start" grid has putting in,that means when this algorithms running
    while not open_set.empty():
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                
        #在open_set裡面,彈出對低 f_score的值,也就是在open_set.put((0, count, start))的 "start" 這個位置的值        
        current = open_set.get()[2] 
        open_set_hash.remove(current) #make sure there is no any duplicates
        #if the current grid as end grid, reconstruct path 
        if current == end:
            reconstruct_path(came_from, end, draw)
            end.make_end()
            return True
        # for all neighbors of current grid
        for neighbor in current.neighbors:
            temp_g_score = g_score[current] + 1 #assume all edges are one
            #assume campare with "temp_g_score: 1" and "g_score[neighbor:infinity", of course "1" < "infinity" 
            if temp_g_score < g_score[neighbor]: 
                #Updating the the value of "parent" of this neighbor from "None"
                came_from[neighbor] = current
                #Updating the g score of neighbor
                g_score[neighbor] = temp_g_score 
                #Updating the f score of neighbor
                f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open()
        draw()

        if current != start:
            current.make_closed()

    return False


#Making grid
def make_grid(rows, width):
    grid = []
    gap = width // rows # width divided by rows equals gap
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot = Spot(i, j, gap, rows) # spot equals new spot
            grid[i].append(spot)

    return grid

def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GRAY, (0, i * gap), (width, i * gap)) #Drawing horizontal line
        for j in range(rows):
            pygame.draw.line(win, GRAY, (j * gap, 0), (j * gap, width)) #Drawing vertical line

# Main drawing function
def draw(win, grid, rows, width):
    win.fill(WHITE)

    for row in grid:
        for spot in row:
            spot.draw(win)

    draw_grid(win, rows, width)
    pygame.display.update()

def get_clicked_pos(pos, rows, width):
    gap = width // rows
    y,x = pos

    row = y // gap
    col = x // gap

    return row, col

#Drawing the route when completing finding the shorest distance
def reconstruct_path(came_from, current, draw):
    while current in came_from:
        current = came_from[current]
        current.make_path()
        draw()

def main(win, width):
    ROWS = 50 #Setting there is 50 units in one line
    grid = make_grid(ROWS, width) #Obtaining the number of grid by calling make_grid method invoking arguments

    start = None
    end = None
    #This boolean 'run' is only for while loop
    run = True
    started = False
    while run:
        draw(win, grid, ROWS, width)
        #for event in all events from the user that be registered into an event queue
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                #Boolean run will set False
                run = False
            if started: #If the program is running, it should not be disturbed by the user
                continue

            if pygame.mouse.get_pressed()[0]: # LEFT Button
                #Storing the location of grid which mouse pointed
                pos = pygame.mouse.get_pos()
                #calculate the value of row, col where mouse pointed
                row, col = get_clicked_pos(pos, ROWS, width)
                
                spot = grid[row][col]
                if not start and spot != end:
                    start = spot
                    start.make_start()
                elif not end and spot != start:
                    end = spot
                    end.make_end()
                elif spot != end and spot != start:
                    spot.make_barrier()

            elif pygame.mouse.get_pressed()[2]: # RIGHT Button
                #Storing the location of grid which mouse pointed
                pos = pygame.mouse.get_pos()
                #calculate the value of row, col where mouse pointed
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]
                spot.reset()
                if spot == start:
                    start = None
                elif spot == end:
                    end = None

            if event.type == pygame.KEYDOWN:     
                if event.key == pygame.K_SPACE and start and end: #'start' and 'end' has pointed
                    for row in grid:
                        for spot in row:
                            spot.update_neighbors(grid)
                    #call the algorithm
                    algorithm(lambda: draw(win, grid, ROWS, width), grid, start, end)

                if event.key == pygame.K_c:
                    start = None
                    end = None
                    grid = make_grid(ROWS, width)

    pygame.quit()

    
main(WIN, WIDTH) #Executing the main program